<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/colab_cucat_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cu_cat demo

(not so risky)

In [ ]:
# !pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
# !pip install git+https://github.com/graphistry/pygraphistry.git@cuCat
# !pip install -U git+https://github.com/graphistry/cu-cat.git@cu-cat #@0.01.0

In [1]:
import cuml
cuml.__version__

'22.10.00a+119.gb30ef85c1'

In [2]:
!echo "0.01.0" > VERSION.txt
!cp VERSION.txt /usr/local/lib/python3.8/dist-packages/cu_cat/

In [3]:
# from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

/usr/local/lib/python3.8/dist-packages/cu_cat/__init__.py


In [4]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)


In [5]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key
graphistry.__version__


'0.28.6+17.g21dac04'

In [6]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
# !pip install dirty_cat
import dirty_cat

### ask HN

In [ ]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# df = askHN.sample(1000,replace=True) # set smaller if you want to test a minibatch 
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()

# g1a = g.featurize(feature_engine='cuCat',memoize=False)
g1a = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat')

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cucat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         24512419 function calls (24455828 primitive calls) in 113.449 seconds

   Ordered by: internal time
   List reduced from 2487 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       13   35.883    2.760   51.176    3.937 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:984(_multiplicative_update_h_smallfast)
      242   10.592    0.044   19.350    0.080 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:932(_multiplicative_update_h)
        9    9.809    1.090    9.810    1.090 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       36    7.886    0.219    7.886    0.219 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
        1    7.176    7.176    7.404    7.404 /usr/local/lib/python3.8/dist-packages/cuml/internals/api_decorators.py:397(inner_with_setters)
       21    4.920    0.234   12.281    0.585 /usr/local/lib/python3.8/dist-packages/sklearn/feature_extr

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


# g1b = g.featurize(feature_engine='dirty_cat',memoize=False)
g1b = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat')

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

# with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

         35746123 function calls (35683329 primitive calls) in 161.237 seconds

   Ordered by: internal time
   List reduced from 2498 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      764   44.261    0.058   93.230    0.122 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
4183584/4182799   41.336    0.000   51.035    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      360   15.234    0.042   37.327    0.104 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
        9    8.568    0.952    8.568    0.952 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
  3786716    5.616    0.000   46.662    0.000 <__array_function__ internals>:177(dot)
       21    4.458    0.212   11.217    0.534 /usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:1184(_count_vocab)
    15120 

In [ ]:
g1a.plot()

In [ ]:
g1b.plot()

### 20newsgroups

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 1000
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
g2a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
g2b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g2a.plot()

In [ ]:
g2b.plot()

### winlogs

In [13]:
# !wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')

winlogs=winlogs.sample(100000,replace=False).iloc[:,:10]

In [8]:
# import cupy as cp
# del g,g2,g3
# cp._default_memory_pool.free_all_blocks()

In [9]:
# from importlib import reload
# reload(cu_cat)
# import cu_cat
# print(cu_cat.__file__)

In [ ]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g3a = g.featurize(feature_engine='cu_cat',memoize=False)
# g3a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_cucat_100k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [42]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()


# g3b = g.featurize(feature_engine='dirty_cat',memoize=False)
g3b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_dirtycat_50k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         31951570 function calls (31788559 primitive calls) in 116.870 seconds

   Ordered by: internal time
   List reduced from 1963 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
3655414/3653726   23.330    0.000   28.512    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     3095   22.475    0.007   64.168    0.021 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
     2765   11.887    0.004   31.699    0.011 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
   165456    5.149    0.000   15.205    0.000 /usr/local/lib/python3.8/dist-packages/scipy/sparse/compressed.py:136(check_format)
  2936401    5.053    0.000   25.508    0.000 <__array_function__ internals>:177(dot)
       21    4.434    0.211    4.435    0.211 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
   342552    4.382 

In [ ]:
g3a.plot()

In [ ]:
g3b.plot()

### redteam

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))
tdf.RED.sum()

749.0

In [ ]:
g = graphistry.nodes(tdf)
pr = cProfile.Profile()
pr.enable()

g4a = g.featurize(feature_engine='cu_cat',memoize=False)
# g3 = g.umap(engine='cuml',feature_engine='cuCat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)


In [ ]:
g = graphistry.nodes(tdf)
pr = cProfile.Profile()
pr.enable()

g4b = g.featurize(feature_engine='dirty_cat',memoize=False)
# g3 = g.umap(engine='cuml',feature_engine='cuCat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)


In [ ]:
%%time
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g4a = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g4b = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         12162452 function calls (12121835 primitive calls) in 80.919 seconds

   Ordered by: internal time
   List reduced from 2081 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5   18.267    3.653   27.114    5.423 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:984(_multiplicative_update_h_smallfast)
        3   17.220    5.740   17.220    5.740 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    39524    8.155    0.000   10.360    0.000 /usr/local/lib/python3.8/dist-packages/pandas/core/missing.py:60(mask_missing)
42881/42646    7.562    0.000   24.912    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    7.102    7.102    7.115    7.115 /usr/local/lib/python3.8/dist-packages/cuml/internals/api_decorators.py:397(inner_with_setters)
      212    4.661    0.022    4.663    0.022 {built-in method cupy._core._routines_linalg.matmul}
     4282

In [ ]:
g4a.plot()

In [ ]:
g4b.plot()

### CTU-13 malware dataset

In [ ]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



In [ ]:
from inspect import getmodule

In [ ]:
# lets umap the data
g = graphistry.edges(edf, src, dst)

pr = cProfile.Profile()
pr.enable()
g5a = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


In [ ]:
### lets umap the data
g = graphistry.edges(edf, src, dst)
pr = cProfile.Profile()
pr.enable()
g5b = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='dirty_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


In [ ]:
g5a.plot()

In [ ]:
g5b.plot()